In [3]:
import tensorflow as tf
#https://medium.com/%E8%BB%9F%E9%AB%94%E4%B9%8B%E5%BF%83/%E9%81%A9%E5%90%88%E5%A4%A7%E9%87%8F%E8%B3%87%E6%96%99i-o%E7%9A%84%E5%84%B2%E5%AD%98%E6%A0%BC%E5%BC%8F-tfrecord%E7%B0%A1%E4%BB%8B%E8%88%87%E6%93%8D%E4%BD%9C%E6%95%99%E5%AD%B8-cd27e50d51ee
"""
message Person {
    required string name = 1;
    required int32 id = 2;
    optional string email = 3;
}
"""

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        # BytesList won't unpack a string from an EagerTensor.
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [4]:
name = []
gpa = []
rank = []
for i in range(100):
    name_now=('haha'+str(i)).encode('utf-8')
    name.append(name_now)
    gpa.append(3.0+i*0.01)
    rank.append(100-i)

def serialize_example(_name, _gpa, _rank):
    feature = {
        "name": _bytes_feature(_name),
        "gpa": _float_feature(_gpa),
        "rank": _int64_feature(_rank),
        }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))

    return example_proto.SerializeToString() 


with tf.io.TFRecordWriter("example.tfrecords") as writer:
    for _name, _gpa, _rank in zip(name, gpa, rank):
        serialized_example = serialize_example(_name, _gpa, _rank)
        writer.write(serialized_example)

In [5]:
dataset = tf.data.TFRecordDataset('example.tfrecords')

image_feature_description = {
    'name': tf.io.FixedLenFeature([], tf.string),
    'gpa': tf.io.FixedLenFeature([], tf.float32),
    'rank': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_function(example_proto):
    serialized_example = tf.io.parse_single_example(example_proto, image_feature_description)
    return serialized_example


for idx, features in enumerate(dataset):
    print(f"reading student {idx}...")
    parsed_features = _parse_function(features)
    for key in parsed_features:
        print(f"{key}: {parsed_features[key].numpy()}")

reading student 0...
gpa: 3.0
name: b'haha0'
rank: 100
reading student 1...
gpa: 3.009999990463257
name: b'haha1'
rank: 99
reading student 2...
gpa: 3.0199999809265137
name: b'haha2'
rank: 98
reading student 3...
gpa: 3.0299999713897705
name: b'haha3'
rank: 97
reading student 4...
gpa: 3.0399999618530273
name: b'haha4'
rank: 96
reading student 5...
gpa: 3.049999952316284
name: b'haha5'
rank: 95
reading student 6...
gpa: 3.059999942779541
name: b'haha6'
rank: 94
reading student 7...
gpa: 3.069999933242798
name: b'haha7'
rank: 93
reading student 8...
gpa: 3.0799999237060547
name: b'haha8'
rank: 92
reading student 9...
gpa: 3.0899999141693115
name: b'haha9'
rank: 91
reading student 10...
gpa: 3.0999999046325684
name: b'haha10'
rank: 90
reading student 11...
gpa: 3.109999895095825
name: b'haha11'
rank: 89
reading student 12...
gpa: 3.119999885559082
name: b'haha12'
rank: 88
reading student 13...
gpa: 3.130000114440918
name: b'haha13'
rank: 87
reading student 14...
gpa: 3.140000104904175
na

In [6]:
import struct
#https://www.kite.com/python/answers/how-to-read-bytes-from-a-binary-file-in-python
#https://docs.python.org/3/library/struct.html

#uint64 length (little endian)
#uint32 masked_crc32_of_length
#byte   data[length]
#uint32 masked_crc32_of_data
binfile=open('example.tfrecords','rb')
#binfile.seek(70)
#for l in binfile.readlines():
#    c+=len(l)
#    print(l)

# read data length
tmp=binfile.read(8)
length=struct.unpack("<Q",tmp)
print(length)

# skip crc
binfile.seek(4,1)

# read data 
tmp=binfile.read(length[0])
#data=struct.unpack("<Q",tmp)
parsed_features = _parse_function(tmp)
for key in parsed_features:
    print(f"{key}: {parsed_features[key].numpy()}")

# skip crc
binfile.seek(4,1)    
binfile.close()

(53,)
gpa: 3.0
name: b'haha0'
rank: 100


#https://www.kite.com/python/answers/how-to-read-bytes-from-a-binary-file-in-python
#uint64 length
#uint32 masked_crc32_of_length
#byte   data[length]
#uint32 masked_crc32_of_data
binfile=open('example.tfrecords','rb')
c=0
for l in binfile.readlines():
    c+=len(l)
    print(l)
print(c)
binfile.close()

In [25]:
def get_non_sparse_instance_length(binfile):
    tmp=binfile.read(8)
    length=struct.unpack("<Q",tmp)
    print(length)
    record_l=length[0]+16
    return record_l

def non_sparse_random_read(binfile, idx, record_l=-1):
    # calc one record length (B) if not specified
    if(record_l<0):
        # read data length
        print("calc one record length")
        tmp=binfile.read(8)
        length=struct.unpack("<Q",tmp)
        print(length)
        # include the 0,2,3 column
        record_l=length[0]+16
    
    # random read
    binfile.seek(idx*record_l,0)
    tmp=binfile.read(record_l)
    r_data=tmp[12:-4]

    # deserialize data
    parsed_features = _parse_function(r_data)
    for key in parsed_features:
        print(f"{key}: {parsed_features[key].numpy()}")
    
    return parsed_features
    
    

In [26]:
import struct
binfile=open('example.tfrecords','rb')
l=get_non_sparse_instance_length(binfile)
a=non_sparse_random_read(binfile, 3, record_l=l)
print(type(a))
print(a)

(53,)
gpa: 3.0299999713897705
name: b'haha3'
rank: 97
<class 'dict'>
{'gpa': <tf.Tensor: shape=(), dtype=float32, numpy=3.03>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'haha3'>, 'rank': <tf.Tensor: shape=(), dtype=int64, numpy=97>}
